In [22]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import logging
from utils import save_file_list
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import scipy
from scipy.signal import stft
from scipy.fft import fft, ifft, fftfreq

In [38]:
def temporalize(X, time_range, step_size):
    onesec_X = []

    for i in range(len(X)-time_range-step_size):
        print(i)
        seq = X[i*step_size:i*step_size+time_range]

        if len(seq) == time_range:
            onesec_X.append(seq)

        else:
            break

    return onesec_X

In [41]:
path_car1 = './dataset_raw_convert/car1/'
path_car1_lt30t = './dataset_raw_convert/car1_lt_30t/'
path_car1_mt30t = './dataset_raw_convert/car1_mt_30t/'
path_car2 = './dataset_raw_convert/car2/'
path_car2_lt30t = './dataset_raw_convert/car2_lt_30t/'
path_car2_mt30t = './dataset_raw_convert/car2_mt_30t/'

In [42]:
ls_car1 = save_file_list(path_car1)
ls_car1_lt30t = save_file_list(path_car1_lt30t)
ls_car1_mt30t = save_file_list(path_car1_mt30t)
ls_car2 = save_file_list(path_car2)
ls_car2_lt30t = save_file_list(path_car2_lt30t)
ls_car2_mt30t = save_file_list(path_car2_mt30t)

29
26
26
29
26
26


In [43]:
scaler = MinMaxScaler()

df_tot = pd.read_csv(ls_car1[0], sep=',')
arr_sensor_scaled = scaler.fit_transform(df_tot['Sensor'].to_numpy().reshape(-1, 1))
arr_sensor_scaled = arr_sensor_scaled.squeeze(1)

sensor_data_1sec_list = temporalize(arr_sensor_scaled, 1000, 10000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228


In [46]:
sensor_data_1sec_list[0].shape

(1000,)

In [65]:
### ----- Function using scipy library
def plot_fft_1sec(file_ls):
    for idx, file in enumerate(file_ls):
        print(file)

        # if 'car1' in file:
        #     vmax=0.02
        # else:
        #     vmax=0.005

        sr = 1000

        """
        scaler = MinMaxScaler()

        df_tot = pd.read_csv(file, sep=',')
        arr_sensor_scaled = scaler.fit_transform(df_tot['Sensor'].to_numpy().reshape(-1, 1))
        arr_sensor_scaled = arr_sensor_scaled.squeeze(1)
        
        sensor_data_1sec_list = temporalize(arr_sensor_scaled, 1000, 10000)
        """
        df_tot = pd.read_csv(file, sep=',')
        sensor_arr = df_tot['Sensor'].to_numpy()
        
        sensor_data_1sec_list = temporalize(sensor_arr, 1000, 10000)


        for idx, seq in enumerate(sensor_data_1sec_list):
            N = seq.shape[0]  ## # of sample points
            T = 1.0 / sr ## Sample slicing

            yf = fft(seq)
            xf = fftfreq(N, T)[:N//2]
            plt.ioff
            plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]))
            plt.grid()            
            plt.savefig(
                './dataset_raw_convert/'
                + file[22:].split("/")[0]
                + '_figure_1sec/fft/' 
                + file[22:].split("/")[1]
                + '_stft_' + str(idx) + '.jpg', 
                bbox_inches='tight',
                pad_inches=0)
            # plt.show()
            plt.close()           
                
            # plt.show()  

In [66]:
plot_fft_1sec(ls_car1)

./dataset_raw_convert/car1/SB25880_20210723-234831_1.csv
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
./dataset_raw_convert/car1/SB25912_20210724-065449_1.csv


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.